# Sentiment Analysis - Chotot user feedback classification

"**Chotot Nha** is using [Surveygizmo](https://www.surveygizmo.com/), an online survey platform, to get **user feedback** about Chotot Nha.
As a business analysis, we need to analyze the **user's feedback**, which feedback is **negative**, **positive** or neutral. Moreover, Nhà BA wants to know more about **bad feedbacks**, which kind of problem which related to this feedback such as **ad-quality**, **payment method**, **premium features**,... In order to help chotot Nha BA identify the kind of feedback, especially bad ones, we might add an AI-brain to help them classify user feedback, then BA can categorize, analyze feedback easily as their kind"

## Sample Data

In [351]:
import pandas as pd
import numpy as np
import re
data = pd.read_csv("data.csv")

data.head(100)


,data,label
0,Cam on cho tốt.,positive
1,Tốt,positive
2,Tin dag chua dc tot lam.. Len hay mat tim k dc,overall
3,Điều chỉnh hay quy định lại giá bán là ...,ad_quality
4,Tốt,positive
5,Ok,positive
6,Hiện chưa có vấn đề gì! Vì cũng thỉnh thoảng m...,neutral
7,Rất tốt,positive
8,Sửa nội dung đã đăng : không cho sửa danh mục,ad_review
9,Cho tot qua tot,positive


In [352]:
data["label"].unique()

array(['positive', 'overall', 'ad_quality', 'neutral', 'ad_review',
       'ui_ux', 'others', 'payment_method', 'premium_features'],
      dtype=object)

In [353]:
groupby = data.groupby(["label"])
groupby.describe()

data         \
                 count unique   
label                           
ad_quality         460    339   
ad_review          527    417   
neutral            193     65   
others              43     29   
overall             37     25   
payment_method     142    120   
positive           758    457   
premium_features   285    252   
ui_ux              397    283   

                                                                          
                                                                top freq  
label                                                                     
ad_quality                                           bỏ bớt tin rác    3  
ad_review                                       duyệt tin nhanh hơn    8  
neutral                                                       không   82  
others                                           Mong dc sự giúp đỡ    2  
overall           đừng bán thông tin cá nhân người đăng tin cho ...    2  
payment_method              đồng tốt nên mở nạp bằng cạt điện thoại    2  
positive                                                        tốt   99  
premium_features                 quảng cáo đến người dùng nhiều hơn    2  
ui_ux                                        đăng ngày mới cập nhật    4

In [354]:
sampleData = data["data"].sample(20).values
sampleData

array(['chợ_tốt rất tốt rồi',
       'nên để lại tính năng chat giữa ng mua và bán',
       'tôi muốn chợ_tốt duyệt tin nhanh hơn',
       'rất tốt duyệt tin mau lẹ cảm on các thành viên ban quãn trị chợ_tốt',
       'để tiện hơn trong việc đưa tin bạn nên để 1 email cho 2 sdt sử dụng',
       'không', 'phần nạp tiền cần đơn giản dễ hiểu dễ nạp hơn',
       'bây giờ có nhiều người đăng tin lừa đảo quá treo đầu dê bán thịt chó đăng bán nhà thạnh lộc kêu dẫn đi xem hẹn gặp cổng mêtro lê văn khương khi ra thì toàn nhân viên tư vấn dự án long an hoặc khi hen địa điểm ra tới nơi thì gọi không nghe máy',
       'những người cần việc làm thực sự thì hãy đăng tin gọi tuyển toàn từ chối bận',
       'khó đăng tin trên chợ_tốt quá Đầu tư tin đăng nhưng ko được duyệt',
       'Việc thanh toán qua thẻ nội địa vẫn gặp khó trong việc thực hiện',
       'Có chính sách ái ngộ cho người bán.để người bán tích cực đăng hơn',
       'tốt', 'Hỗ trợ duyệt tin nhanh', 'tốt', 'tệ phức tạp khó khăn',
       'c

**It can be seen that user feedback have some particular features:**
* Contain both sign and no sign (chợ tốt and cho tot)
* Imbalance dataset
* We must tokenize chợ tốt to chợ_tốt because "tốt" is the confused keyword
* Number is redundant features

## Data Preprocessing

#### Convert to lowercase

In [355]:
def convertToLower(s):
    return s.lower()

sampleData = list(map(convertToLower, sampleData))
sampleData

['chợ_tốt rất tốt rồi',
 'nên để lại tính năng chat giữa ng mua và bán',
 'tôi muốn chợ_tốt duyệt tin nhanh hơn',
 'rất tốt duyệt tin mau lẹ cảm on các thành viên ban quãn trị chợ_tốt',
 'để tiện hơn trong việc đưa tin bạn nên để 1 email cho 2 sdt sử dụng',
 'không',
 'phần nạp tiền cần đơn giản dễ hiểu dễ nạp hơn',
 'bây giờ có nhiều người đăng tin lừa đảo quá treo đầu dê bán thịt chó đăng bán nhà thạnh lộc kêu dẫn đi xem hẹn gặp cổng mêtro lê văn khương khi ra thì toàn nhân viên tư vấn dự án long an hoặc khi hen địa điểm ra tới nơi thì gọi không nghe máy',
 'những người cần việc làm thực sự thì hãy đăng tin gọi tuyển toàn từ chối bận',
 'khó đăng tin trên chợ_tốt quá đầu tư tin đăng nhưng ko được duyệt',
 'việc thanh toán qua thẻ nội địa vẫn gặp khó trong việc thực hiện',
 'có chính sách ái ngộ cho người bán.để người bán tích cực đăng hơn',
 'tốt',
 'hỗ trợ duyệt tin nhanh',
 'tốt',
 'tệ phức tạp khó khăn',
 'cần hướng dẫn cách nạp tiền để đăng tin',
 'phí đăng tin cao quá nên lấy ch

#### Remove number

In [356]:
def removeNumber(s):
    return re.sub(r"\d+", "", s)

sampleData = list(map(removeNumber, sampleData))
sampleData

['chợ_tốt rất tốt rồi',
 'nên để lại tính năng chat giữa ng mua và bán',
 'tôi muốn chợ_tốt duyệt tin nhanh hơn',
 'rất tốt duyệt tin mau lẹ cảm on các thành viên ban quãn trị chợ_tốt',
 'để tiện hơn trong việc đưa tin bạn nên để  email cho  sdt sử dụng',
 'không',
 'phần nạp tiền cần đơn giản dễ hiểu dễ nạp hơn',
 'bây giờ có nhiều người đăng tin lừa đảo quá treo đầu dê bán thịt chó đăng bán nhà thạnh lộc kêu dẫn đi xem hẹn gặp cổng mêtro lê văn khương khi ra thì toàn nhân viên tư vấn dự án long an hoặc khi hen địa điểm ra tới nơi thì gọi không nghe máy',
 'những người cần việc làm thực sự thì hãy đăng tin gọi tuyển toàn từ chối bận',
 'khó đăng tin trên chợ_tốt quá đầu tư tin đăng nhưng ko được duyệt',
 'việc thanh toán qua thẻ nội địa vẫn gặp khó trong việc thực hiện',
 'có chính sách ái ngộ cho người bán.để người bán tích cực đăng hơn',
 'tốt',
 'hỗ trợ duyệt tin nhanh',
 'tốt',
 'tệ phức tạp khó khăn',
 'cần hướng dẫn cách nạp tiền để đăng tin',
 'phí đăng tin cao quá nên lấy chi 

#### Remove punctuation

In [357]:
def removePunctuation(s):
    s = re.sub(r"[^A-Za-z0-9\u00C0-\u02B8\u037F-\u051D\u1E00-\u1F79 ]+", "", s)
    return s.strip()

sampleData = list(map(removePunctuation, sampleData))
sampleData

['chợtốt rất tốt rồi',
 'nên để lại tính năng chat giữa ng mua và bán',
 'tôi muốn chợtốt duyệt tin nhanh hơn',
 'rất tốt duyệt tin mau lẹ cảm on các thành viên ban quãn trị chợtốt',
 'để tiện hơn trong việc đưa tin bạn nên để  email cho  sdt sử dụng',
 'không',
 'phần nạp tiền cần đơn giản dễ hiểu dễ nạp hơn',
 'bây giờ có nhiều người đăng tin lừa đảo quá treo đầu dê bán thịt chó đăng bán nhà thạnh lộc kêu dẫn đi xem hẹn gặp cổng mêtro lê văn khương khi ra thì toàn nhân viên tư vấn dự án long an hoặc khi hen địa điểm ra tới nơi thì gọi không nghe máy',
 'những người cần việc làm thực sự thì hãy đăng tin gọi tuyển toàn từ chối bận',
 'khó đăng tin trên chợtốt quá đầu tư tin đăng nhưng ko được duyệt',
 'việc thanh toán qua thẻ nội địa vẫn gặp khó trong việc thực hiện',
 'có chính sách ái ngộ cho người bánđể người bán tích cực đăng hơn',
 'tốt',
 'hỗ trợ duyệt tin nhanh',
 'tốt',
 'tệ phức tạp khó khăn',
 'cần hướng dẫn cách nạp tiền để đăng tin',
 'phí đăng tin cao quá nên lấy chi phí t

#### Remove stopword

In [358]:
f = open("stopword.txt", "r").readlines()
stopwords = [s.replace("\n", "") for s in f]
print(stopwords)

def removeStopword(s):
    res = filter(lambda x: x not in stopwords, s.split(" "))
    return " ".join(res)

sampleData = list(map(removeStopword, sampleData))
sampleData

['các', 'cái', 'càng', 'chiếc', 'cho', 'chứ', 'cứ', 'của', 'cùng', 'cũng', 'đã', 'đang', 'đây', 'để', 'đều', 'điều', 'do', 'đó', 'được', 'gì', 'khi', 'không', 'là', 'lại', 'lên', 'lúc', 'mà', 'mỗi', 'này', 'nên', 'nếu', 'ngay', 'như', 'nhưng', 'những', 'nơi', 'nữa', 'qua', 'ra', 'rằng', 'rồi', 'sẽ', 'so', 'tại', 'thì', 'từ', 'và', 'vẫn', 'vậy', 'vì', 'với', 'vừa']


['chợtốt rất tốt',
 'tính năng chat giữa ng mua bán',
 'tôi muốn chợtốt duyệt tin nhanh hơn',
 'rất tốt duyệt tin mau lẹ cảm on thành viên ban quãn trị chợtốt',
 'tiện hơn trong việc đưa tin bạn  email  sdt sử dụng',
 '',
 'phần nạp tiền cần đơn giản dễ hiểu dễ nạp hơn',
 'bây giờ có nhiều người đăng tin lừa đảo quá treo đầu dê bán thịt chó đăng bán nhà thạnh lộc kêu dẫn đi xem hẹn gặp cổng mêtro lê văn khương toàn nhân viên tư vấn dự án long an hoặc hen địa điểm tới gọi nghe máy',
 'người cần việc làm thực sự hãy đăng tin gọi tuyển toàn chối bận',
 'khó đăng tin trên chợtốt quá đầu tư tin đăng ko duyệt',
 'việc thanh toán thẻ nội địa gặp khó trong việc thực hiện',
 'có chính sách ái ngộ người bánđể người bán tích cực đăng hơn',
 'tốt',
 'hỗ trợ duyệt tin nhanh',
 'tốt',
 'tệ phức tạp khó khăn',
 'cần hướng dẫn cách nạp tiền đăng tin',
 'phí đăng tin cao quá lấy chi phí nhà mạng miễn phí tin đăng hoặc tính phí thấp ng dùng',
 'duyệt tin lâu quá khó',
 '']

#### Tokenization

In [359]:
def tokenization(s):
    s = s.replace("chợ tốt", "chợ_tốt")
    s = s.replace("cho tot", "cho_tot")
    return s
sampleData = list(map(tokenization, sampleData))
sampleData

['chợtốt rất tốt',
 'tính năng chat giữa ng mua bán',
 'tôi muốn chợtốt duyệt tin nhanh hơn',
 'rất tốt duyệt tin mau lẹ cảm on thành viên ban quãn trị chợtốt',
 'tiện hơn trong việc đưa tin bạn  email  sdt sử dụng',
 '',
 'phần nạp tiền cần đơn giản dễ hiểu dễ nạp hơn',
 'bây giờ có nhiều người đăng tin lừa đảo quá treo đầu dê bán thịt chó đăng bán nhà thạnh lộc kêu dẫn đi xem hẹn gặp cổng mêtro lê văn khương toàn nhân viên tư vấn dự án long an hoặc hen địa điểm tới gọi nghe máy',
 'người cần việc làm thực sự hãy đăng tin gọi tuyển toàn chối bận',
 'khó đăng tin trên chợtốt quá đầu tư tin đăng ko duyệt',
 'việc thanh toán thẻ nội địa gặp khó trong việc thực hiện',
 'có chính sách ái ngộ người bánđể người bán tích cực đăng hơn',
 'tốt',
 'hỗ trợ duyệt tin nhanh',
 'tốt',
 'tệ phức tạp khó khăn',
 'cần hướng dẫn cách nạp tiền đăng tin',
 'phí đăng tin cao quá lấy chi phí nhà mạng miễn phí tin đăng hoặc tính phí thấp ng dùng',
 'duyệt tin lâu quá khó',
 '']

#### Steming

In [360]:
def steming(s):
    return s

## Append sentence without sign

In [361]:
import unicodedata

def removeAccent(s):
#     s = s.decode('utf-8')
    s = re.sub(u'Đ', 'D', s)
    s = re.sub(u'đ', 'd', s)
    return unicodedata.normalize('NFKD', str(s)).encode('ASCII', 'ignore').decode()

addedData = data
for index, row in data.iterrows():
    addedData.loc[index] = [removeAccent(row['data']), row["label"]]
data = data.append(added_data)
data


,data,label
0,Cam on cho tot.,positive
1,Tot,positive
2,Tin dag chua dc tot lam.. Len hay mat tim k dc,overall
3,Dieu chinh hay quy dinh lai gia ban la gia the...,ad_quality
4,Tot,positive
5,Ok,positive
6,Hien chua co van de gi! Vi cung thinh thoang m...,neutral
7,Rat tot,positive
8,Sua noi dung da dang : khong cho sua danh muc,ad_review
9,Cho tot qua tot,positive


## Create Model

In [362]:
def cleanData(s):
    s = converToLower(s)
    s = removeNumber(s)
    s = removePunctuation(s)
    s = removeStopword(s)
    s = tokenization(s)
    s = steming(s)
    return s

In [363]:
def tokenizer(s):
    return s.split()

#### train test split

In [364]:
labelMapping = {
    "ad_quality": 0,
    "ad_review": 1,
    "neutral": 2,
    "others": 3,
    "overall": 4,
    "payment_method": 5,
    "positive": 6,
    "premium_features": 7,
    "ui_ux": 8
}

data = data.dropna()

y = data["label"].map(labelMapping)
X = data["data"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [365]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stopwords,
                        tokenizer=tokenizer,
                        preprocessor=cleanData,ngram_range=(1,2))

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0,multi_class="multinomial",solver="lbfgs"))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<function clean..., penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [366]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
prediction = clf.predict(X_test)

In [367]:
print(classification_report(y_true=y_test, y_pred=prediction))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       206
           1       0.93      0.95      0.94       210
           2       0.93      0.94      0.94        72
           3       1.00      0.62      0.76        13
           4       1.00      0.13      0.24        15
           5       0.87      0.85      0.86        55
           6       0.95      0.95      0.95       282
           7       0.95      0.91      0.93       121
           8       0.89      0.94      0.91       163

   micro avg       0.93      0.93      0.93      1137
   macro avg       0.94      0.81      0.83      1137
weighted avg       0.93      0.93      0.92      1137



In [368]:
accuracy_score(y_test, prediction)

0.9287598944591029